滑动平均（影子值）：记录每一个参数一段时间内过往值的平均，增加了模型的泛化性。
针对所有的参数：w和b。像是给参数加了影子，参数变化，影子缓慢跟随。

$$\text{影子初始值}=\text{参数初始值}$$
$$\text{影子}=\text{衰减率}*\text{影子} + (1 - \text{衰减率}) * \text{参数值}$$
$$\text{衰减率} = min\{\text{MOVE_AVERAGE_DECAY}, \frac{1 + \text{轮数}}{10 + \text{轮数}}\}$$
$$\text{MOVE_AVERAGE_DECAY} = 0.99$$

```python
ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
```
 - MOVING_AVERAGE_DECAY:衰减率
 - global_step:当前轮数

`ema_op = ema.apply([])`的括号里面是更新列表，每次运行`sess.run(ema_op)`会对更新列表中的参数求取滑动平均。实际应用中会自动将所有的参数汇总为列表，如下：`ema_op = ema.apply(tf.trainable_variables())`

```python
with tf.control_dependencies([train_step, ema_op]):
    train = tf.no_op(name="train")
ema.average(参数名)  # 返回默写参数的平均值
```

In [1]:
import tensorflow as tf

In [12]:
w = tf.Variable(0., tf.float32)
global_step = tf.Variable(0, trainable=False)
EXPONENTIAL_MOVING_AVERAGE = 0.99
ema = tf.train.ExponentialMovingAverage(EXPONENTIAL_MOVING_AVERAGE, global_step)
ema_op = ema.apply(tf.trainable_variables())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([w, ema.average(w)]))
    
    for i in range(10):
        sess.run(tf.assign_add(w, 1))
        sess.run(tf.assign_add(global_step, 1))
        sess.run(ema_op)
        print(sess.run([global_step, w, ema.average(w)]))

[0.0, 0.0]
[1, 1.0, 0.8181818]
[2, 2.0, 1.7045455]
[3, 3.0, 2.6013987]
[4, 4.0, 3.5004995]
[5, 5.0, 4.4002]
[6, 6.0, 5.3000875]
[7, 7.0, 6.2000413]
[8, 8.0, 7.1000204]
[9, 9.0, 8.0000105]
[10, 10.0, 8.900005]
